# Example: Multiple Tables
In this example, I show how doctable can be used with multiple inter-related tables to perform queries which automatically merge different aspects of your dataset when you use `.select()`. By integrating these relations into the schema, your database can automatically maintain consistency between tables by deleting irrelevant elements when their relations disappear. There are two important features of any multi-table schema using doctable:

(1) Set the foreign_keys=True in the original doctable or ConnectEngine constructor. Otherwise sqlalchemy will not enable.

(2) Use the "foreignkey" column type to set the constraint, probably with the onupdate and ondelete keywords specifiied.

In this example I'll create interrelated tables for authors and their books.

In [1]:
import datetime

import sys
sys.path.append('..')
import doctable

In [2]:
@doctable.schema
class Author:
    __slots__ = []
    id: int = doctable.IDCol()
    name: str = doctable.Col(unique=True)
    fav_color: str = doctable.Col()
    date_updated: datetime.datetime = doctable.UpdatedCol()


class Authors(doctable.DocTable):
    _tabname_ = 'authors'
    _schema_ = Author

adb = Authors(target=':memory:', foreign_keys=True)
#adb.execute('pragma foreign_keys=ON')
adb

In [3]:
@doctable.schema
class Book:
    __slots__ = []
    id: int = doctable.IDCol()
    title: str = doctable.Col(unique=True)
    authname: str = doctable.Col()
    date_updated: datetime.datetime = doctable.UpdatedCol()

class Books(doctable.DocTable):
    _tabname_ = 'books'
    _schema_ = Book

    _constraints_ = (
        doctable.Constraint('foreignkey', ('authname',), ('authors.name',), onupdate="CASCADE", ondelete="CASCADE"),
    )
    _indices_ = (
        doctable.Index('ind_authtitle', 'title', 'authname', unique=True),
    )

bdb = Books(engine=adb.engine)
bdb

In [4]:
# see that both are registered with the engine metadata
adb.engine.tables.keys()

dict_keys(['authors', 'books'])

In [5]:
# define a test dataset
collection = (
    ('Devin Cornell', 'green', 'The Case of Austerity'),
    ('Devin Cornell', 'green', 'Gender Stereotypes'),
    ('Devin Cornell', 'green', 'Colombian Politics'),
    ('Pierre Bourdieu', 'orange', 'Distinction'),
    ('Pierre Bourdieu', 'orange', 'Symbolic Power'),
    ('Jean-Luc Picard', 'red', 'Enterprise Stories'),
)

In [6]:
for auth, color, title in collection:
    adb.insert({'name':auth, 'fav_color': color}, ifnotunique='ignore')
    bdb.insert({'authname':auth, 'title': title}, ifnotunique='ignore')
adb.count(), bdb.count()

(3, 6)

In [7]:
adb.head()

,id,name,fav_color,date_updated
0,1,Devin Cornell,green,2021-09-01 13:40:22.868675
1,2,Pierre Bourdieu,orange,2021-09-01 13:40:22.871401
2,3,Jean-Luc Picard,red,2021-09-01 13:40:22.873444


In [8]:
bdb.head(10)

,id,title,authname,date_updated
0,1,The Case of Austerity,Devin Cornell,2021-09-01 13:40:22.869716
1,2,Gender Stereotypes,Devin Cornell,2021-09-01 13:40:22.870460
2,3,Colombian Politics,Devin Cornell,2021-09-01 13:40:22.871065
3,4,Distinction,Pierre Bourdieu,2021-09-01 13:40:22.871958
4,5,Symbolic Power,Pierre Bourdieu,2021-09-01 13:40:22.872896
5,6,Enterprise Stories,Jean-Luc Picard,2021-09-01 13:40:22.873984


## Joint Select Statements
You can perform joins by using select queries with column objects from different tables.

In [9]:
# this is a left join
bdb.select(['title', adb['name'], adb['fav_color']], where=bdb['authname']==adb['name'], as_dataclass=False)

[('Colombian Politics', 'Devin Cornell', 'green'),
 ('Distinction', 'Pierre Bourdieu', 'orange'),
 ('Enterprise Stories', 'Jean-Luc Picard', 'red'),
 ('Gender Stereotypes', 'Devin Cornell', 'green'),
 ('Symbolic Power', 'Pierre Bourdieu', 'orange'),
 ('The Case of Austerity', 'Devin Cornell', 'green')]

In [10]:
# with tables reversed, still returns same output
adb.select(['name', bdb['title']], where=adb['name']==bdb['authname'], as_dataclass=False)

[('Devin Cornell', 'Colombian Politics'),
 ('Pierre Bourdieu', 'Distinction'),
 ('Jean-Luc Picard', 'Enterprise Stories'),
 ('Devin Cornell', 'Gender Stereotypes'),
 ('Pierre Bourdieu', 'Symbolic Power'),
 ('Devin Cornell', 'The Case of Austerity')]

## Cascade deletion
See now that by deleting the author "Devin Cornell", we also removed the corresponding rows in the book table.

In [11]:
adb.delete(where=adb['name']=='Devin Cornell')

In [12]:
adb.head()

,id,name,fav_color,date_updated
0,2,Pierre Bourdieu,orange,2021-09-01 13:40:22.871401
1,3,Jean-Luc Picard,red,2021-09-01 13:40:22.873444


In [13]:
bdb.head(10)

,id,title,authname,date_updated
0,4,Distinction,Pierre Bourdieu,2021-09-01 13:40:22.871958
1,5,Symbolic Power,Pierre Bourdieu,2021-09-01 13:40:22.872896
2,6,Enterprise Stories,Jean-Luc Picard,2021-09-01 13:40:22.873984
